In [2]:
import geopandas as gpd
import rasterio
from rasterio.sample import sample_gen
import numpy as np

# === File paths ===
junctions_fp = r"C:\Users\jmsch\OneDrive\Documenten\Studie\Civiele Techniek\Environmental Engineering\Year 2\Afstuderen\NBS Nakuru Kenia\2. QGIS + Wflow\SWMM\SWD network\updated_junctions_SWMM.gpkg"       # Your input point file
junctions_layer = "updated_junctions_SWMM"   # Layer name in GPKG
dem_fp = r"C:\Users\jmsch\OneDrive\Documenten\Studie\Civiele Techniek\Environmental Engineering\Year 2\Afstuderen\NBS Nakuru Kenia\2. QGIS + Wflow\DATA\2023-03-07 LIDAR DEM\Block 2\6m_DEM_TIFF_NAKURU_TOWN\DEM_MERGED_COMPLETE_WGS84.tif"                    # Your DEM raster file
output_fp = r"C:\Users\jmsch\OneDrive\Documenten\Studie\Civiele Techniek\Environmental Engineering\Year 2\Afstuderen\NBS Nakuru Kenia\2. QGIS + Wflow\SWMM\SWD network\updated_junctions_SWMM.gpkg"  # Output file

# === Load junction points ===
gdf = gpd.read_file(junctions_fp)

# Ensure CRS matches between DEM and vector
with rasterio.open(dem_fp) as dem:
    if gdf.crs != dem.crs:
        gdf = gdf.to_crs(dem.crs)
    
    # Sample DEM at each point
    coords = [(x,y) for x, y in zip(gdf.geometry.x, gdf.geometry.y)]
    elevations = list(dem.sample(coords))
    elevations = [float(val[0]) if val[0] != dem.nodata else np.nan for val in elevations]

# === Add or update fields ===
gdf["Elevation"] = elevations
gdf["MaxDepth"] = 1.0
gdf["InitDepth"] = 0.0
gdf["SurDepth"] = 0.0
gdf["Aponded"] = 10.0
gdf["Name"] = gdf.index.map(lambda i: f"J_{i}")
gdf["Annotation"] = ""

# === Save updated file ===
gdf.to_file(output_fp, layer="updated_junctions", driver="GPKG")

print("✅ Junction layer updated and saved to:", output_fp)


✅ Junction layer updated and saved to: C:\Users\jmsch\OneDrive\Documenten\Studie\Civiele Techniek\Environmental Engineering\Year 2\Afstuderen\NBS Nakuru Kenia\2. QGIS + Wflow\SWMM\SWD network\updated_junctions_SWMM.gpkg


In [5]:
import geopandas as gpd
from shapely.geometry import LineString, Point
from sklearn.neighbors import NearestNeighbors
import numpy as np

# === File paths ===
conduits = gpd.read_file(r"C:\Users\jmsch\OneDrive\Documenten\Studie\Civiele Techniek\Environmental Engineering\Year 2\Afstuderen\NBS Nakuru Kenia\2. QGIS + Wflow\SWMM\SWD network\updated_conduits_SWMM.gpkg")
junctions = gpd.read_file(r"C:\Users\jmsch\OneDrive\Documenten\Studie\Civiele Techniek\Environmental Engineering\Year 2\Afstuderen\NBS Nakuru Kenia\2. QGIS + Wflow\SWMM\SWD network\updated_junctions_SWMM.gpkg")
#junctions_layer = "updated_junctions"
output_fp = r"C:\Users\jmsch\OneDrive\Documenten\Studie\Civiele Techniek\Environmental Engineering\Year 2\Afstuderen\NBS Nakuru Kenia\2. QGIS + Wflow\SWMM\SWD network\updated_conduits_SWMM_new.gpkg"

# Ensure both are in a projected CRS
if conduits.crs.is_geographic:
    conduits = conduits.to_crs(epsg=32736)  # Replace with your UTM zone
if junctions.crs != conduits.crs:
    junctions = junctions.to_crs(conduits.crs)

import geopandas as gpd
import numpy as np
from scipy.spatial import cKDTree

# === Prepare junction lookup ===
junctions["coords"] = junctions.geometry.apply(lambda geom: (geom.x, geom.y))
junction_tree = cKDTree(junctions["coords"].tolist())
junction_coords = np.array(junctions["coords"].tolist())
junction_names = junctions["Name"].values
junction_elevs = junctions["Elevation"].values

# === Determine flow direction for each conduit ===
from_nodes = []
to_nodes = []

for line in conduits.geometry:
    start_xy = (line.coords[0][0], line.coords[0][1])
    end_xy = (line.coords[-1][0], line.coords[-1][1])

    _, start_idx = junction_tree.query(start_xy)
    _, end_idx = junction_tree.query(end_xy)

    elev_start = junction_elevs[start_idx]
    elev_end = junction_elevs[end_idx]

    name_start = junction_names[start_idx]
    name_end = junction_names[end_idx]

    if elev_start >= elev_end:
        from_nodes.append(name_start)
        to_nodes.append(name_end)
    else:
        from_nodes.append(name_end)
        to_nodes.append(name_start)

# === Assign new FID and Name to conduits ===
conduits["fid"] = range(len(conduits))
conduits["Name"] = conduits["fid"].apply(lambda x: f"C_{x}")

# === Assign FromNode and ToNode ===
conduits["FromNode"] = from_nodes
conduits["ToNode"] = to_nodes

# Retain all other columns, keeping the original structure
keep_cols = ['fid', 'Name', 'Annotation']
for col in keep_cols:
    if col not in conduits.columns:
        conduits[col] = np.nan  # Fill if missing

# Reorder (optional)
cols = keep_cols + [c for c in conduits.columns if c not in keep_cols]
conduits = conduits[cols]


# === Save output ===
conduits.to_file(output_fp, layer="updated_conduits", driver="GPKG")
print(f"✅ Conduits saved to: {output_fp}")


✅ Conduits saved to: C:\Users\jmsch\OneDrive\Documenten\Studie\Civiele Techniek\Environmental Engineering\Year 2\Afstuderen\NBS Nakuru Kenia\2. QGIS + Wflow\SWMM\SWD network\updated_conduits_SWMM_new.gpkg
